In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re #regular expressions

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/messy-text-data/dates.txt


This is the first assignment in the course Applied Text Mining in Python from Michigan University. In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data.
Each line of the dates.txt file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats. The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates.

Here is a list of some of the variants you might encounter in this dataset:

* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:

* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.
* With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices. This Series should be sorted by a tie-break sort in the format of ("extracted date", "original row number").

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [2]:
doc = []
with open('/kaggle/input/messy-text-data/dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df = pd.DataFrame(df, columns=['text'])
df.head(10)

,text
0,03/25/93 Total time of visit (in minutes):\n
1,6/18/85 Primary Care Doctor:\n
2,sshe plans to move as of 7/8/71 In-Home Servic...
3,7 on 9/27/75 Audit C Score Current:\n
4,2/6/96 sleep studyPain Treatment Pain Level (N...
5,.Per 7/06/79 Movement D/O note:\n
6,"4, 5/18/78 Patient's thoughts about current su..."
7,10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8,3/7/86 SOS-10 Total Score:\n
9,(4/10/71)Score-1Audit C Score Current:\n


In this part of the code we create the different patterns to iterate over all the data, then I create a list for all the patterns to iterate over all the pattern from 1 to 10

In [3]:

pattern1 = r'(?P<month>\d{1,2})[-/\s](?P<day>\d{1,2})[-/\s](?P<year>\d{2,4})'   # (dd/mm/yy) or (dd/mm/yyyy) or (dd-mm-yy) or (dd-mm-yyyy)
pattern2 = r'(?P<day>\d{1,2})\s(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,\s](?P<year>\d{2,4})' # dd month(letters) yyyy
pattern3 = r'(?P<day>\d{1,2})\s(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[.,]\s(?P<year>\d{4})' # dd month(letters)(, or .) yyyy
pattern4 = r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s(?P<day>\d{1,2}),\s(?P<year>\d{4})' #month(letters) dd, yyyy
pattern5 = r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.\s(?P<day>\d{1,2}),\s(?P<year>\d{4})' #month(letters). dd, yyyy
pattern6 = r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s(?P<day>\d{1,2})\s(?P<year>\d{4})' #month(letters) dd yyyy
pattern7 = r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s(?P<year>\d{4})' #month(letters) yyyy
pattern8 = r'(?P<month>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[,]\s(?P<year>\d{4})' #month(letters), yyyy
pattern9 = r'(?P<month>\d{1,2})[-\/](?P<year>\d{4})' #mm(/ or -)yyyy
pattern10 = r'(?P<year>\d{4})'

patterns = [pattern1, pattern2, pattern3, pattern4, pattern5, pattern6, pattern7, pattern8, pattern9, pattern10]

Create empty lists for append the corresponding month, day and year

In [4]:
#create three list to save the month, days and year
months = []
days = []
years = []


In [5]:
for i in range(len(df)): 
    text = df.iloc[i, 0]  # Extraer el texto
    found = False  # Variable para controlar si se ha encontrado una coincidencia
    
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            months.append(match.group('month')) if 'month' in match.groupdict() else months.append("1")
            days.append(match.group('day')) if 'day' in match.groupdict() else days.append("1")
            years.append(match.group('year'))
            found = True
            break  # Salir del bucle interior si se encuentra una coincidencia
    
    if not found:  #This part of code was used to check if I need to add new patterns or modify any
        print(text)
        months.append(None)
        days.append(None)
        years.append(None)

In [6]:
'''indice_none = months.index(None)  #Check if there is any None value
indice_none'''

'indice_none = months.index(None)  #Check if there is any None value\nindice_none'

In [7]:
month_mapping = {
    'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06',
    'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
}

# Transform months names as numbers and change the type for all the list
months_numerical = [month_mapping[month] if len(month) > 2 else month for month in months]
months_2 = list(map(int, months_numerical)) #this is the final list with int values

days_2 = list(map(int, days))

years = ['19' + year if len(year) < 4 else year for year in years]
years_2 = list(map(int, years))


Add to the dataframe

In [8]:
df["Day"] = days_2
df["Month"] = months_2
df["Year"] = years_2

and sort

In [9]:
df.sort_values(by = ["Year", "Month", "Day"], inplace = True)
df.reset_index(inplace = True)
ordered_series = pd.Series(df["index"])

This is the final list, the minimin date is 04/10/197 and the max date is 11/01/2016

In [10]:
df

,index,text,Day,Month,Year
0,9,(4/10/71)Score-1Audit C Score Current:\n,10,4,1971
1,84,5/18/71 Total time of visit (in minutes):\n,18,5,1971
2,2,sshe plans to move as of 7/8/71 In-Home Servic...,8,7,1971
3,53,7/11/71 SOS-10 Total Score:\n,11,7,1971
4,28,9/12/71 [report_end]\n,12,9,1971
...,...,...,...,...,...
495,427,6e. monitor pt's depressive experience in cont...,1,5,2016
496,141,30 May 2016 SOS-10 Total Score:\n,30,5,2016
497,186,13 Oct 2016 Primary Care Doctor:\n,13,10,2016
498,161,19 Oct 2016 Communication with referring physi...,19,10,2016
